# Tarea 1: Conceptos Básicos y Programación en Python

**Universidad Autonóma de Nuevo León** \\
Facultad de Ciencias Fisíco Matemáticas

\\

**Introducción al Aprendizaje Automático** \\
Lic. Ángel Adrián Domínguez Lozano 

\\

**Información del Equipo \#14**

* 1845533 Sofía Alejandra Gaytán Díaz 
* 1842834 Juan Daniel Morón Guajardo 
* 1862464 Humberto Gerardo Peña Páez

\\

## Entregable 2: Implementación de Stochastic Gradient Descent para regresión logística

### Objetivo:
Que el alumno tenga un punto de partida para consultar temas relacionados al aprendizaje automático.

### Instrucciones:

Considerando un problema de clasificación binaria, usando un escalar continuo x para predecir la variable binaria y (y=0 o y=1)
Se utiliza la función sigmoide f(x) = 1/( 1 +  exp(-x*m-b)) para ajustar la distribución de y con respecto a x. 
La función de pérdida utilizada usualmente es la binary cross-entrophy. 

Realiza una implementación de stochastic gradient descent para minimizar el binary cross-entropy. 

Los puntos a realizar son los siguientes:

* Calcular el gradiente de la función de pérdida con respecto a m y b. 
* Generar un conjunto aleatorio de datos de la siguiente manera:
  * 100 valores aleatorios uniformemente distribuidos para x entre -10 y 10 
  * Asignar y a cada valor de x usando el criterio: \\
    Generar un número aleatorio uniforme R entre 0 y 1:

\\

$$
\text{Si }  x \geq 2 \text{ y } R \leq 0.9 \rightarrow y=1 \\
\text{Si } x \geq 2 \text{ y } R>0.9 \rightarrow  y=0 \\
\text{Si } x < 2 \text{ y } R \leq 0.9 \rightarrow  y=0 \\
\text{Si } x < 2 \text{ y } R>0.9 \rightarrow  y=1 \\
$$

\\

* Implementar SGD y revisar su desempeño con los datos simulados.

### Desarrollo del Programa

#### Librerías
Se importaron e implementaron las siguientes librerias para desarrollar el codigo:
* Numpy
* Pandas
* Plotly Express

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px

#### Generación de los valores de $X$
Se declaro una variable $N$ para determinar la cantidad de valores del arreglo $x$ para despues ser instanciado como un arreglo de una dimensión de $N$ números aleatorios que van desde $-10$ a $10$ en base a una distribución uniforme.

In [ ]:
#Cantidad de valores aleatorios para x
N = 100

#N valores aleatorios uniformemente distribuidos para x entre -10 y 10
x = np.random.uniform(low=-10,high=10, size=(N,1))

#### Generación de los valores de $Y$
Se instancia un arreglo vacio $y$ para despues ser llenado en un ciclo donde se tomara en cuenta distintas situaciones en base al valor del arreglo $x$ y un valor del arreglo $R$, arreglo con $N$ números aleatorios de entre 0 y 1.

En el ciclo, se genera un conjunto aleatorio de datos para $y$ en base a las siguientes restricciones:

\\

$$
\text{Si }  x \geq 2 \text{ y } R \leq 0.9 \rightarrow y=1 \\
\text{Si } x \geq 2 \text{ y } R>0.9 \rightarrow  y=0 \\
\text{Si } x < 2 \text{ y } R \leq 0.9 \rightarrow  y=0 \\
\text{Si } x < 2 \text{ y } R>0.9 \rightarrow  y=1 \\
$$

In [ ]:
#Inicialización de y
y = np.zeros(shape=(N,1), dtype="int")
R = np.random.uniform(low=0,high=1,size=(N,1))

#Generar un conjunto aleatorio de datos para y
for i in range(0,N):
  if x[i]>=2 and R[i]<=0.9:
    y[i] = 1
  elif x[i]>=2 and R[i]>0.9:
    y[i] = 0
  elif x[i]<2 and R[i]<=0.9:
    y[i] = 0
  elif x[i]<2 and R[i]>0.9:
    y[i] = 1


#### Visualización del comportamiento de los valores $X$ y $Y$
Para ver mejor el comportamiento de los valores de $x$ y $y$ se crea un **Data Frame** en base a estos arreglos, para despues ser visualizados en un gráfico scatter.

In [ ]:
#Generación del Data Frame
df = pd.DataFrame({'x':np.squeeze(x),'y':np.squeeze(y)})
df = df.sort_values(by=['x'])

#Visualización del gráfico
px.scatter(df,x='x',y='y')

#### Función de Perdida
La función de perdida implementada es:
$$
L(y,\hat{y})= \frac{1}{N} \sum_{i=1}^{N}{y\log{(\hat{y})} + (1 - y)\log{(1-\hat{y}})}
$$

In [ ]:
def loss(y,yhat):
  return  -np.mean((y * np.log(yhat)) + ((1-y) * np.log(1-yhat)))

#### Calcular la gradiente en base a $m$
La gradiente se saca en base a la derivada parcial de $L(y,\hat{y})$ en base a $m$, obteniendo la siguiente ecuación:

$$
\frac { x(ye^{(-mx-b)} + (y-1))}{(1 + e^{-mx-b})}
$$

Y su promedio, donde empleamos la función mean de la librería numpy para esto.

In [ ]:
def g_m(x,y,m,b):
  return -np.mean(x*((np.exp(-m*x-b)*y)+(y-1))/(1+np.exp(-m*x-b)))

#### Calcular la gradiente en base a $m$
La gradiente se saca en base a la derivada parcial de $L(y,\hat{y})$ en base a $b$, obteniendo la siguiente ecuación:

$$
\frac { ye^{(-mx-b)} + (y-1)}{(1 + e^{-mx-b})}
$$

Y su promedio, donde empleamos la función mean de la librería numpy para esto.

In [ ]:
def g_b(x,y,m,b):
  return -np.mean(((np.exp(-m*x-b)*y)+(y-1))/(1+np.exp(-m*x-b)))

#### Calcular la función de perdida $J$
Se calcula la función de perdida que se quiere **minimizar** en base a los datos $m$ y $b$ que se obtienen del algoritmo. Es de ayuda para obtener más rapido el resultado en base a la tolerancia y la diferencia entre la funcion perdida de los valores anteriores con el actual.

In [ ]:
def J(m,b,x,y):
  yhat = np.zeros(shape=(N,1))
  for i in range(N):
    yhat[i] = 1 / (1 + np.exp(-x[i]*m-b))
  return loss(y,yhat)

#### Implementación del algoritmo SGD
Para la implementación del algoritmo se utilizan los siguientes parametros:
* **Iteraciones:** Número de iteraciones del algoritmo. En nuestro caso, un número grande, $10,000$.
* **Lr:** Indicador del seguimiento del gradiente para no caer en maximizaciones. En nuestro caso, se determino un número cercano a cero, $0.01$.
* **Batch:** Número de elementos que tendra el subconjunto o muestra de valores $x$ y valores $y$ para obtener un resultado más especifico. 
* **Tolerancia:** Indicador de mejoramiento en base a la función de perdida ($J$). En nuestro caso, se experimento con el caso más extremo y especifico, determinando la tolerancia en $0\%$ en base a observaciones mencionadas al final.



In [ ]:
#Iteraciones
iteraciones = 10000

#Lr
lr = 0.01

#Batch
batch = 10

#Tolerancia
tol = 0

Se inicializan las variables $m$ y $b$ como un número aleatorio entre $0$ y $10$. 

In [ ]:
m =  np.random.randint(0,10)
b = np.random.randint(0,10)

print(f"m inicial: {m}")
print(f"b inicial: {b}")

m inicial: 2
b inicial: 7


Se prosigue a implementar el algoritmo las $K$ iteraciones que se determinaron en un principios, donde se van a obtener $m$ y $b$ en base a los gradientes y, en este caso, se detendra el programa y se escogeran las mejores tomando en cuenta el mejoramiento en base a la tolerancia que se determino al principio y la funcion de perdida. 

In [ ]:
i = 0

while i <= iteraciones:
  Jant = J(m,b,x,y)
  sub = np.random.randint(low=0,high=N,size=batch)
  gr_b = g_b(x[sub,:],y[sub,:],m,b)
  gr_m = g_m(x[sub,:],y[sub,:],m,b)
  m = m - lr*gr_m
  b = b - lr*gr_b
  Jnew = J(m,b,x,y)
  if np.abs(Jnew-Jant) <= tol:
    i = iteraciones
  i = i + 1

Finalizando con el algoritmo se obtienen los valores de $m$ y $b$, y se podra determinar la función de perdida en base a estos valores empleando la función $J$ que se declaro anteriormente.

In [ ]:
print(f"m: {m}")
print(f"b: {b}")
print(f"Función de perdida: {J(m,b,x,y)}")

m: 0.5154193135545778
b: -1.7437061366689892
Función de perdida: 0.26687317422317636


#### Visualización de la Sigmoide
Obteniendo estos dos valores, se determina la función sigmoide dada por $\hat{y}$ que cumple con los puntos dados por $x$ y $y$.

In [ ]:
yhat = np.zeros(shape=(N,1))

for i in range(N):
  yhat[i] = 1 / (1 + np.exp(-x[i]*m-b))

dfhat = pd.DataFrame({'x':np.squeeze(x),
                      'yhat':np.squeeze(yhat)})
dfhat = dfhat.sort_values(by=['x'])
dfhat = dfhat.melt(id_vars=['x'])

px.line(dfhat,x='x',y='value',color='variable')

#### Observaciones
Durante la experimentación con el algoritmo se visualizo un comportamiento muy diverso en base a la determinación de la tolerancia. Por ejemplo, en un principio se determino la tolerancia con un valor mayor a $0$, en este caso, $0.1$, donde se obtenia una visualización de una sigmoide con caracteristicas muy diferentes a una sigmoide normal.

In [ ]:
iteraciones = 10000
lr = 0.01
batch = 10
tol = 0.1

m =  np.random.randint(0,10)
b = np.random.randint(0,10)

i = 0
while i <= iteraciones:
  Jant = J(m,b,x,y)
  sub = np.random.randint(low=0,high=N,size=batch)
  gr_b = g_b(x[sub,:],y[sub,:],m,b)
  gr_m = g_m(x[sub,:],y[sub,:],m,b)
  m = m - lr*gr_m
  b = b - lr*gr_b
  Jnew = J(m,b,x,y)
  if np.abs(Jnew-Jant) <= tol:
    i = iteraciones
  i = i + 1

yhat = np.zeros(shape=(N,1))

for i in range(N):
  yhat[i] = 1 / (1 + np.exp(-x[i]*m-b))

dfhat = pd.DataFrame({'x':np.squeeze(x),
                      'yhat':np.squeeze(yhat)})
dfhat = dfhat.sort_values(by=['x'])
dfhat = dfhat.melt(id_vars=['x'])

px.line(dfhat,x='x',y='value',color='variable')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: RuntimeWarning:

divide by zero encountered in log

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: RuntimeWarning:

invalid value encountered in multiply



Por ello, se fue experimentando hasta encontrar una visualización consistente, donde se visualizo que esto pasaba cuando la tolerancia se acercaba a $0$, determinando asi la tolerancia en un $0\%$.

In [ ]:
iteraciones = 10000
lr = 0.01
batch = 10
tol = 0.000099

m =  np.random.randint(0,10)
b = np.random.randint(0,10)

i = 0
while i <= iteraciones:
  Jant = J(m,b,x,y)
  sub = np.random.randint(low=0,high=N,size=batch)
  gr_b = g_b(x[sub,:],y[sub,:],m,b)
  gr_m = g_m(x[sub,:],y[sub,:],m,b)
  m = m - lr*gr_m
  b = b - lr*gr_b
  Jnew = J(m,b,x,y)
  if np.abs(Jnew-Jant) <= tol:
    i = iteraciones
  i = i + 1

yhat = np.zeros(shape=(N,1))

for i in range(N):
  yhat[i] = 1 / (1 + np.exp(-x[i]*m-b))

dfhat = pd.DataFrame({'x':np.squeeze(x),
                      'yhat':np.squeeze(yhat)})
dfhat = dfhat.sort_values(by=['x'])
dfhat = dfhat.melt(id_vars=['x'])

px.line(dfhat,x='x',y='value',color='variable')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: RuntimeWarning:

divide by zero encountered in log

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: RuntimeWarning:

invalid value encountered in multiply

